# One-to-Many and Many-to-Many Joins - Lab

## Introduction

In this lab, you'll practice your knowledge of one-to-many and many-to-many relationships!

## Objectives

You will be able to:
* Explain one-to-many and many-to-many joins as well as implications for the size of query results
* Query data using one-to-many and many-to-many joins

## One-to-Many and Many-to-Many Joins
<img src='images/Database-Schema.png' width="600">

## Connect to the Database

In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Employees and their Office (a One-to-One join)

Return a DataFrame with all of the employees including their first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [8]:
sql_str = """SELECT e.firstName
             , e.lastName
             , o.city AS office_city
             , COALESCE(o.state, o.territory) AS office_state_or_terr
             FROM employees e
             JOIN offices o USING (officeCode)
             ORDER BY e.firstName, e.lastName;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,firstName,lastName,office_city,office_state_or_terr
0,Andy,Fixter,Sydney,
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY
5,George,Vanauf,NYC,NY
6,Gerard,Bondur,Paris,
7,Gerard,Hernandez,Paris,
8,Jeff,Firrelli,San Francisco,CA
9,Julie,Firrelli,Boston,MA


## Customers and their Orders (a One-to-Many join)

Return a DataFrame with all of the customer contacts (first and last names) along with details for each of the customers' order numbers, order dates, and statuses.

In [11]:
sql_str = """SELECT c.contactfirstName AS customer_firstname
             , c.contactlastName AS customer_lastname
             , o.orderNumber
             , o.orderDate
             , o.status AS orderStatus
             FROM customers c
             JOIN orders o USING (customerNumber)
             ORDER BY customer_lastname, customer_firstname, o.orderDate;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,customer_firstname,customer_lastname,orderNumber,orderDate,orderStatus
0,Paolo,Accorti,10280,2004-08-17,Shipped
1,Paolo,Accorti,10293,2004-09-09,Shipped
2,Rachel,Ashworth,10110,2003-03-18,Shipped
3,Rachel,Ashworth,10306,2004-10-14,Shipped
4,Rachel,Ashworth,10332,2004-11-17,Shipped
...,...,...,...,...,...
321,Julie,Young,10145,2003-08-25,Shipped
322,Julie,Young,10189,2003-11-18,Shipped
323,Julie,Young,10367,2005-01-12,Resolved
324,Mary,Young,10154,2003-10-02,Shipped


## Customers and their Payments (another One-to-Many join)

Return a DataFrame with all of the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. Sort these results in descending order by the payment amount. 

In [12]:
sql_str = """SELECT c.contactfirstName AS customer_firstname
             , c.contactlastName AS customer_lastname
             , p.paymentDate
             , p.amount AS paymentAmount
             FROM customers c
             JOIN payments p USING (customerNumber)
             ORDER BY paymentAmount DESC;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,customer_firstname,customer_lastname,paymentDate,paymentAmount
0,Diego,Freyre,2005-03-18,120166.58
1,Diego,Freyre,2004-12-31,116208.40
2,Susan,Nelson,2003-08-15,111654.40
3,Eric,Natividad,2003-12-26,105743.00
4,Susan,Nelson,2005-03-05,101244.59
...,...,...,...,...
268,Carine,Schmitt,2004-12-18,1676.14
269,Pascale,Cartrain,2003-04-19,1627.56
270,Jonas,Bergulfsen,2003-10-28,1491.38
271,Pascale,Cartrain,2003-08-22,1128.20


## Orders, Order details, and Product Details (a Many-to-Many Join)

Return a DataFrame with all of the customer contacts (first and last names) along with the product names, quantities, and date ordered for each of the customers and each of their orders. Sort these in descending order by the order date.

> Note: This will require joining 4 tables! This can be tricky! Give it a shot, and if you're still stuck, turn to the next section where you'll see how to write subqueries that can make complex queries such as this much simpler!

In [14]:
sql_str = """SELECT c.contactfirstName AS customer_firstname
             , c.contactlastName AS customer_lastname
             , o.orderNumber
             , o.orderDate
             , p.productName
             , od.quantityOrdered
             FROM customers c
             JOIN orders o USING (customerNumber)
             JOIN orderdetails od USING (orderNumber)
             JOIN products p USING (productCode)
             ORDER BY o.orderDate, o.orderNumber DESC;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,customer_firstname,customer_lastname,orderNumber,orderDate,productName,quantityOrdered
0,Dorothy,Young,10100,2003-01-06,1917 Grand Touring Sedan,30
1,Dorothy,Young,10100,2003-01-06,1911 Ford Town Car,50
2,Dorothy,Young,10100,2003-01-06,1932 Alfa Romeo 8C2300 Spider Sport,22
3,Dorothy,Young,10100,2003-01-06,1936 Mercedes Benz 500k Roadster,49
4,Roland,Keitel,10101,2003-01-09,1932 Model A Ford J-Coupe,25
...,...,...,...,...,...,...
2991,Diego,Freyre,10424,2005-05-31,1958 Setra Bus,49
2992,Diego,Freyre,10424,2005-05-31,1940 Ford Pickup Truck,54
2993,Diego,Freyre,10424,2005-05-31,1939 Cadillac Limousine,26
2994,Diego,Freyre,10424,2005-05-31,1996 Peterbilt 379 Stake Bed with Outrigger,44


## Summary

In this lab, you practiced your knowledge of one-to-many and many-to-many relationships!